# MELTS
  
Versions of MELTS implemented are:  
- MELTS v. 1.0.2 ➞ (rhyolite-MELTS, Gualda et al., 2012)  
- MELTS v. 1.1.0 ➞ (rhyolite-MELTS + new CO<sub>2</sub>, works at the ternary minimum)  
- MELTS v. 1.2.0 ➞ (rhyolite-MELTS + new H<sub>2</sub>O + new CO<sub>2</sub>)  
- pMELTS v. 5.6.1

## Initialize tools and packages that are required to execute this notebook.

In [ ]:
from thermoengine import equilibrate
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Create a MELTS v 1.0.2 instance.
Rhyolite-MELTS version 1.0.2 is the default model.

In [ ]:
melts = equilibrate.MELTSmodel()

### Optional: Generate some information about the implemented model.

In [ ]:
oxides = melts.get_oxide_names()
phases = melts.get_phase_names()
#print (oxides)
#print (phases)

## Required: Input initial composition of the system (liquid), in wt% or grams of oxides. 
Mid-Atlantic ridge MORB composition

In [ ]:
feasible = melts.set_bulk_composition({'SiO2':  48.68, 
                                       'TiO2':   1.01, 
                                       'Al2O3': 17.64, 
                                       'Fe2O3':  0.89,
                                       'Cr2O3':  0.0425, 
                                       'FeO':    7.59, 
                                       'MnO':    0.0,
                                       'MgO':    9.10, 
                                       'NiO':    0.0, 
                                       'CoO':    0.0,
                                       'CaO':   12.45, 
                                       'Na2O':   2.65, 
                                       'K2O':    0.03, 
                                       'P2O5':   0.08, 
                                       'H2O':    0.2})

## Optional: Suppress phases that are not required in the simulation.

In [ ]:
b = melts.get_phase_inclusion_status()
melts.set_phase_inclusion_status({'Nepheline':False, 'OrthoOxide':False})
a = melts.get_phase_inclusion_status()
for phase in b.keys():
    if b[phase] != a[phase]:
        print ("{0:<15s} Before: {1:<5s} After: {2:<5s}".format(phase, repr(b[phase]), repr(a[phase])))

## Compute the equilibrium state at some specified T (°C) and P (MPa).
Print status of the calculation.

In [ ]:
output = melts.equilibrate_tp(1220.0, 100.0, initialize=True)
(status, t, p, xmlout) = output[0]
print (status, t, p)

### Summary output of equilibrium state ...

In [ ]:
melts.output_summary(xmlout)
dict = melts.get_dictionary_of_affinities(xmlout, sort=True)
for phase in dict:
    (affinity, formulae) = dict[phase]
    if affinity < 10000.0:
        print ("{0:<20s} {1:10.2f} {2:<60s}".format(phase, affinity, formulae))

### Obtain default set of fractionation coefficients (retain liquids, fractionate solids and fluids)

In [ ]:
frac_coeff = melts.get_dictionary_of_default_fractionation_coefficients()
print (frac_coeff)

## Run the sequence of calculations along a T, P=constant path:
Output is sent to an Excel file and plotted in the notebook

In [ ]:
number_of_steps = 40
t_increment_of_steps = -5.0
p_increment_of_steps = 0.0

plotOxides = ['SiO2', 'FeO', 'MgO', 'CaO', 'Na2O']
# matplotlib colors b : blue, g : green, r : red, c : cyan, m : magenta, y : yellow, k : black, w : white.
plotColors = [ 'ro', 'bo', 'go', 'co', 'mo']

wb = melts.start_excel_workbook_with_sheet_name(sheetName="Summary")
melts.update_excel_workbook(wb, xmlout)

n = len(plotOxides)
xPlot = np.zeros(number_of_steps+1)
yPlot = np.zeros((n, number_of_steps+1))
xPlot[0] = t
for i in range (0, n):
    oxides = melts.get_composition_of_phase(xmlout, 'Liquid')
    yPlot[i][0] = oxides[plotOxides[i]]

plt.ion()
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim([min(t, t+t_increment_of_steps*number_of_steps), max(t, t+t_increment_of_steps*number_of_steps)])
ax.set_ylim([0., 100.])
graphs = []
for i in range (0, n):
    graphs.append(ax.plot(xPlot, yPlot[i], plotColors[i]))
handle = []
for (graph,) in graphs:
    handle.append(graph)
ax.legend(handle, plotOxides, loc='upper left')

for i in range (1, number_of_steps):
    # fractionate phases
    frac_output = melts.fractionate_phases(xmlout, frac_coeff)
    output = melts.equilibrate_tp(t+t_increment_of_steps, p+p_increment_of_steps, initialize=True)
    (status, t, p, xmlout) = output[0]
    print ("{0:<30s} {1:8.2f} {2:8.2f}".format(status, t, p))
    xPlot[i] = t
    for j in range (0, n):
        oxides = melts.get_composition_of_phase(xmlout, 'Liquid')
        yPlot[j][i] = oxides[plotOxides[j]]
    j = 0
    for (graph,) in graphs:
        graph.set_xdata(xPlot)
        graph.set_ydata(yPlot[j])
        j = j + 1
    fig.canvas.draw()
    melts.update_excel_workbook(wb, xmlout)



In [ ]:
# Temporarily avoid writing excel file to disk
# melts.write_excel_workbook(wb, "MELTSv102summary.xlsx")